In [ ]:
!pip install requests

In [ ]:
import pandas as pd
import sqlite3
import requests

# URL of the SQLite database file
db_url = 'https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite'
db_file = 'Chinook_Sqlite.sqlite'

# Download the database file
response = requests.get(db_url)
with open(db_file, 'wb') as f:
    f.write(response.content)

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

# List all tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(tables_query, conn)

# Display the list of tables
print("Available tables:")
print(tables)

# Close the connection
# conn.close()

Available tables:
             name
0           Album
1          Artist
2        Customer
3        Employee
4           Genre
5         Invoice
6     InvoiceLine
7       MediaType
8        Playlist
9   PlaylistTrack
10          Track


DOCUMENTATION
https://github.com/lerocha/chinook-database

Questions!
1. List all albums by a specific artist.
2. Count the number of albums for each artist.
3. List all tracks in a specific album.
4. Find the total sales per customer.
5. Find the customer who has spent the most money.
6. Find the best-selling track.
7. Find the most popular genre based on the number of tracks
8. Calculate the total revenue generated by each track.
9. List the employees who are also sales agents.
10. Find the total number of sales by each sales agent.

Advanced!
1. Finding the Top 5 Customers with the Highest Number of Different Tracks Purchased
Objective: Determine the top 5 customers who have purchased the highest number of different tracks, along with the number of distinct tracks they have purchased.

2. Identifying the Most Popular Genre in Each Country Based on Sales

Objective: For each country, identify the most popular genre based on the total sales. The result should include the country, genre, and total sales amount for that genre.

In [ ]:
# # Connect to the SQLite database
# conn = sqlite3.connect(db_file)

# # Query the database and load it into a DataFrame
# query = 'SELECT * FROM Track'
# df = pd.read_sql_query(query, conn)

# # Close the connection
# # conn.close()

# # Display the DataFrame
# df.head()

In [ ]:
def execute_query(conn, sql_querry):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(sql_querry)
    except Error as e:
        print(e)

    print(f"Returned values: {c.fetchall()}")
    return c.fetchall()

In [ ]:
import sqlite3

def extract_extended_schema(db_path):
    # Connect to the database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Enable foreign key support
    cursor.execute("PRAGMA foreign_keys = ON;")

    # Get all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    schema = {}

    # For each table, get the schema (columns and types)
    for table in tables:
        table_name = table[0]
        # Get table columns
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()

        # Get foreign keys
        cursor.execute(f"PRAGMA foreign_key_list({table_name});")
        foreign_keys = cursor.fetchall()

        schema[table_name] = {
            "columns": columns,
            "foreign_keys": foreign_keys
        }

    conn.close()
    return schema

# Example: use the path to your SQLite database (replace with the actual path to the Chinook database)
db_path = 'Chinook_Sqlite.sqlite'
schema = extract_extended_schema(db_path)

# Print out the extracted schema
for table, details in schema.items():
    print(f"Table: {table}")
    print("Columns:")
    for column in details["columns"]:
        print(f" - {column[1]} ({column[2]})")

    if details["foreign_keys"]:
        print("Foreign Keys:")
        for fk in details["foreign_keys"]:
            print(f" - From column {fk[3]} to {fk[2]}.{fk[4]}")
    else:
        print("No Foreign Keys")
    print("\n")

Table: Album
Columns:
 - AlbumId (INTEGER)
 - Title (NVARCHAR(160))
 - ArtistId (INTEGER)
Foreign Keys:
 - From column ArtistId to Artist.ArtistId


Table: Artist
Columns:
 - ArtistId (INTEGER)
 - Name (NVARCHAR(120))
No Foreign Keys


Table: Customer
Columns:
 - CustomerId (INTEGER)
 - FirstName (NVARCHAR(40))
 - LastName (NVARCHAR(20))
 - Company (NVARCHAR(80))
 - Address (NVARCHAR(70))
 - City (NVARCHAR(40))
 - State (NVARCHAR(40))
 - Country (NVARCHAR(40))
 - PostalCode (NVARCHAR(10))
 - Phone (NVARCHAR(24))
 - Fax (NVARCHAR(24))
 - Email (NVARCHAR(60))
 - SupportRepId (INTEGER)
Foreign Keys:
 - From column SupportRepId to Employee.EmployeeId


Table: Employee
Columns:
 - EmployeeId (INTEGER)
 - LastName (NVARCHAR(20))
 - FirstName (NVARCHAR(20))
 - Title (NVARCHAR(30))
 - ReportsTo (INTEGER)
 - BirthDate (DATETIME)
 - HireDate (DATETIME)
 - Address (NVARCHAR(70))
 - City (NVARCHAR(40))
 - State (NVARCHAR(40))
 - Country (NVARCHAR(40))
 - PostalCode (NVARCHAR(10))
 - Phone (NVARCHA

In [ ]:
!pip install graphviz

In [ ]:
from graphviz import Digraph

def visualize_extended_schema(schema):
    dot = Digraph(comment="Chinook Database Schema")

    # Add nodes for each table with their columns
    for table, details in schema.items():
        label = f"{table}\n"
        for col in details["columns"]:
            label += f"{col[1]} ({col[2]})\n"  # Column name and type
        dot.node(table, label=label, shape="box")

    # Add edges for foreign keys
    for table, details in schema.items():
        for fk in details["foreign_keys"]:
            from_table = table
            to_table = fk[2]  # The table it references
            from_column = fk[3]  # Column in the current table
            to_column = fk[4]  # Column in the referenced table
            dot.edge(from_table, to_table, label=f"{from_column} -> {to_column}")

    return dot

# Visualize the schema with foreign keys
dot = visualize_extended_schema(schema)
dot.render('chinook_database_schema', format='png', cleanup=False)  # Renders as PNG

'chinook_database_schema.png'

1. List all albums by a specific artist.
Replace 'AC/DC' with the desired artist’s name.

In [ ]:
execute_query(conn, """
SELECT Album.Title
FROM Album
JOIN Artist ON Album.ArtistId = Artist.ArtistId
WHERE Artist.Name = 'AC/DC';
""")

Returned values: [('For Those About To Rock We Salute You',), ('Let There Be Rock',)]


[]

2. Count the number of albums for each artist.

In [ ]:
execute_query(conn, """
SELECT Artist.Name, COUNT(Album.AlbumId) AS AlbumCount
FROM Album
JOIN Artist ON Album.ArtistId = Artist.ArtistId
GROUP BY Artist.Name;
""")

Returned values: [('AC/DC', 2), ('Aaron Copland & London Symphony Orchestra', 1), ('Aaron Goldberg', 1), ('Academy of St. Martin in the Fields & Sir Neville Marriner', 1), ('Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner', 1), ('Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair', 1), ('Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart', 1), ('Accept', 2), ('Adrian Leaper & Doreen de Feis', 1), ('Aerosmith', 1), ('Aisha Duo', 1), ('Alanis Morissette', 1), ('Alberto Turco & Nova Schola Gregoriana', 1), ('Alice In Chains', 1), ('Amy Winehouse', 2), ('Anne-Sophie Mutter, Herbert Von Karajan & Wiener Philharmoniker', 1), ('Antal Doráti & London Symphony Orchestra', 1), ('Antônio Carlos Jobim', 2), ('Apocalyptica', 1), ('Aquaman', 1), ('Audioslave', 3), ('BackBeat', 1), ('Barry Wordsworth & BBC Concert Orchestra', 1), ('Battlestar Galactica', 2), ('Battlestar Galactica (Classic)', 1), ('Berliner Philharm

[]

3. List all tracks in a specific album.

In [ ]:
execute_query(conn, """
SELECT Track.Name
FROM Track
JOIN Album ON Track.AlbumId = Album.AlbumId
WHERE Album.Title = 'For Those About To Rock We Salute You';
""")

Returned values: [('For Those About To Rock (We Salute You)',), ('Put The Finger On You',), ("Let's Get It Up",), ('Inject The Venom',), ('Snowballed',), ('Evil Walks',), ('C.O.D.',), ('Breaking The Rules',), ('Night Of The Long Knives',), ('Spellbound',)]


[]

4. Find the total sales per customer.


In [ ]:
execute_query(conn, """
SELECT Customer.FirstName, Customer.LastName, SUM(Invoice.Total) AS TotalSpent
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
GROUP BY Customer.CustomerId;
""")

Returned values: [('Luís', 'Gonçalves', 39.62), ('Leonie', 'Köhler', 37.620000000000005), ('François', 'Tremblay', 39.62), ('Bjørn', 'Hansen', 39.62), ('František', 'Wichterlová', 40.620000000000005), ('Helena', 'Holý', 49.620000000000005), ('Astrid', 'Gruber', 42.62), ('Daan', 'Peeters', 37.62), ('Kara', 'Nielsen', 37.620000000000005), ('Eduardo', 'Martins', 37.620000000000005), ('Alexandre', 'Rocha', 37.620000000000005), ('Roberto', 'Almeida', 37.62), ('Fernanda', 'Ramos', 37.620000000000005), ('Mark', 'Philips', 37.620000000000005), ('Jennifer', 'Peterson', 38.620000000000005), ('Frank', 'Harris', 37.62), ('Jack', 'Smith', 39.620000000000005), ('Michelle', 'Brooks', 37.620000000000005), ('Tim', 'Goyer', 38.620000000000005), ('Dan', 'Miller', 39.62), ('Kathy', 'Chase', 37.62), ('Heather', 'Leacock', 39.62), ('John', 'Gordon', 37.62), ('Frank', 'Ralston', 43.62), ('Victor', 'Stevens', 42.62), ('Richard', 'Cunningham', 47.620000000000005), ('Patrick', 'Gray', 37.620000000000005), ('Jul

[]

5. Find the customer who has spent the most money.

In [ ]:
execute_query(conn, """
SELECT Customer.FirstName, Customer.LastName, SUM(Invoice.Total) AS TotalSpent
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
GROUP BY Customer.CustomerId
ORDER BY TotalSpent DESC
LIMIT 1;
""")

Returned values: [('Helena', 'Holý', 49.620000000000005)]


[]

6. Find the best-selling track.

In [ ]:
execute_query(conn, """
SELECT Track.Name, SUM(InvoiceLine.UnitPrice) AS TotalSaleValue
FROM Track
JOIN InvoiceLine ON Track.TrackId = InvoiceLine.TrackId
GROUP BY Track.TrackId
ORDER BY TotalSaleValue DESC
LIMIT 8;
""")

Returned values: [('The Woman King', 3.98), ('The Fix', 3.98), ('Walkabout', 3.98), ('Hot Girl', 3.98), ('Gay Witch Hunt', 3.98), ("Phyllis's Wedding", 3.98), ('How to Stop an Exploding Man', 3.98), ('Pilot', 3.98)]


[]

7. Find the most popular genre based on the number of tracks.

In [ ]:
execute_query(conn, """
SELECT Genre.Name, COUNT(Track.TrackId) AS TotalTracks
FROM Genre
JOIN Track ON Genre.GenreId = Track.GenreId
GROUP BY Genre.GenreId
ORDER BY TotalTracks DESC
LIMIT 1;
""")

Returned values: [('Rock', 1297)]


[]

8. Calculate the total revenue generated by each track.

In [ ]:
execute_query(conn, """
SELECT Track.Name, SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS TotalRevenue
FROM Track
JOIN InvoiceLine ON Track.TrackId = InvoiceLine.TrackId
GROUP BY Track.TrackId;
""")

Returned values: [('For Those About To Rock (We Salute You)', 0.99), ('Balls to the Wall', 1.98), ('Fast As a Shark', 0.99), ('Restless and Wild', 0.99), ('Princess of the Dawn', 0.99), ('Put The Finger On You', 0.99), ('Inject The Venom', 1.98), ('Snowballed', 1.98), ('Evil Walks', 0.99), ('Breaking The Rules', 0.99), ('Night Of The Long Knives', 0.99), ('Spellbound', 0.99), ('Go Down', 0.99), ('Dog Eat Dog', 0.99), ('Problem Child', 0.99), ('Overdose', 1.98), ("Hell Ain't A Bad Place To Be", 0.99), ('Love In An Elevator', 0.99), ('Rag Doll', 0.99), ('What It Takes', 0.99), ("Janie's Got A Gun", 0.99), ('Amazing', 0.99), ('Blind Man', 0.99), ('Deuces Are Wild', 1.98), ('Angel', 0.99), ("Livin' On The Edge", 0.99), ('All I Really Want', 0.99), ('You Oughta Know', 0.99), ('Right Through You', 0.99), ('Forgiven', 0.99), ('You Learn', 0.99), ('Not The Doctor', 1.98), ('Wake Up', 0.99), ('Sea Of Sorrow', 0.99), ('Bleed The Freak', 0.99), ("I Can't Remember", 0.99), ("It Ain't Like That", 0

[]

9. List the employees who are also sales agents.


In [ ]:
execute_query(conn, """
SELECT Employee.FirstName, Employee.LastName
FROM Employee
WHERE Employee.Title LIKE 'Sales Support%';
""")

Returned values: [('Jane', 'Peacock'), ('Margaret', 'Park'), ('Steve', 'Johnson')]


[]

10. Find the total number of sales by each sales agent.

In [ ]:
execute_query(conn, """
SELECT Employee.FirstName, Employee.LastName, COUNT(Invoice.InvoiceId) AS TotalSales
FROM Employee
JOIN Customer ON Employee.EmployeeId = Customer.SupportRepId
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
GROUP BY Employee.EmployeeId;
""")

Returned values: [('Jane', 'Peacock', 146), ('Margaret', 'Park', 140), ('Steve', 'Johnson', 126)]


[]

ADV_1. Find the Top 5 Customers with the Highest Number of Different Tracks Purchased.

In [ ]:
execute_query(conn, """
SELECT Customer.FirstName, Customer.LastName, COUNT(DISTINCT InvoiceLine.TrackId) AS DistinctTracks
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
GROUP BY Customer.CustomerId
ORDER BY DistinctTracks DESC
LIMIT 5;
""")

Returned values: [('Manoj', 'Pareek', 38), ('Luis', 'Rojas', 38), ('Diego', 'Gutiérrez', 38), ('Mark', 'Taylor', 38), ('Steve', 'Murray', 38)]


[]

ADV_2.
Identifying the Most Popular Genre in Each Country Based on Sales.

In [ ]:
execute_query(conn, """
SELECT Country, Genre.Name, COUNT(InvoiceLine.TrackId) AS TotalSales
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Genre ON Track.GenreId = Genre.GenreId
GROUP BY Customer.Country, Genre.GenreId
ORDER BY Country, TotalSales DESC;
""")

Returned values: [('Argentina', 'Alternative & Punk', 9), ('Argentina', 'Rock', 9), ('Argentina', 'Latin', 8), ('Argentina', 'Metal', 7), ('Argentina', 'Easy Listening', 2), ('Argentina', 'Jazz', 2), ('Argentina', 'Soundtrack', 1), ('Australia', 'Rock', 22), ('Australia', 'Metal', 8), ('Australia', 'Heavy Metal', 3), ('Australia', 'Reggae', 2), ('Australia', 'Latin', 2), ('Australia', 'Blues', 1), ('Austria', 'Rock', 15), ('Austria', 'Metal', 7), ('Austria', 'TV Shows', 4), ('Austria', 'R&B/Soul', 4), ('Austria', 'Classical', 2), ('Austria', 'Latin', 2), ('Austria', 'Jazz', 2), ('Austria', 'Drama', 1), ('Austria', 'Pop', 1), ('Belgium', 'Rock', 21), ('Belgium', 'Alternative & Punk', 14), ('Belgium', 'R&B/Soul', 2), ('Belgium', 'Metal', 1), ('Brazil', 'Rock', 81), ('Brazil', 'Latin', 53), ('Brazil', 'Metal', 15), ('Brazil', 'Alternative & Punk', 7), ('Brazil', 'Classical', 6), ('Brazil', 'Reggae', 6), ('Brazil', 'Blues', 6), ('Brazil', 'Soundtrack', 4), ('Brazil', 'R&B/Soul', 3), ('Braz

[]